In [1]:
# Importar librerías
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from concurrent.futures import ThreadPoolExecutor
import random

In [2]:
# Definir la función para inicializar el navegador
def get_driver():
    
    # Extraer el directorio actual
    current_dir = os.getcwd()

    # Definir el path del driver de Chrome
    chrome_driver_path = os.path.join(current_dir, "chromedriver")

    # Definir el user-agent que deseas utilizar
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/112.0.5615.138 Safari/537.36"

    # Definir las opciones del navegador
    options = Options()
    options.add_argument("--headless") # Correr el navegador en modo headless
    options.add_argument(f"user-agent={user_agent}")

    # Definir el servicio de Chrome
    service = Service(executable_path=chrome_driver_path)

    # Inicializar el navegador
    driver = webdriver.Chrome(service=service, options=options)
    
    return driver

In [3]:
# Definir la función para extraer los page_ids únicos de los assets
def get_unique_page_ids(url):

    # Inicia una instancia del navegador web
    driver = get_driver()

    # Navegar a la URL
    driver.get(url)

    # Esperar 3 segundos para que cargue la página
    time.sleep(3)

    # Definir el número de pixeles a scrollear
    scroll_distance = 600
    
    # Definir el set vació para almacenar los page_ids únicos
    unique_page_ids = set()

    # Definir la función para extraer los page_ids
    def extract_data():
        
        # Extraer el código HTML
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extraer los MarketCards
        market_cards = soup.find_all("a", class_="MarketCard__wrapper")
        
        # Extraer los page_ids
        page_ids = [market_card["href"] for market_card in market_cards]
        
        return page_ids

    while True:
        total_scroll_distance = 0
        page_height = driver.execute_script("return document.body.scrollHeight")

        while total_scroll_distance < page_height:
            driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
            total_scroll_distance += scroll_distance
            time.sleep(2)

            page_height = driver.execute_script("return document.body.scrollHeight")

            page_ids = extract_data()
            unique_page_ids.update(page_ids)

        try:
            load_more_button = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.XPATH, '//button[contains(@class, "CustomButton--tertiary") and contains(text(), "Load More")]'))
            )
            load_more_button.click()
            
            # Esperar 2 segundos para que cargue la página
            time.sleep(2)
        
        # Si no hay más botones de "Load More", romper el ciclo
        except Exception as e:
            break
    
    # Imprimir el número de page_ids únicos extraídos
    print(f"Número de page_ids únicos: {len(unique_page_ids)}")
    
    # Cerrar el navegador
    driver.quit()
    
    return list(unique_page_ids)

In [4]:
# Definir la URL de búsqueda
url = "https://wax.atomichub.io/market?collection_name=dolpchainwax&data:text.rarity=legendary&order=desc&sort=created&state=1&symbol=WAX"

# Obtener los page_ids únicos
unique_page_ids = get_unique_page_ids(url)

Número de page_ids únicos: 35


In [5]:
# Configurar la URL base de assets de AtomicHub
base_url = 'https://wax.atomichub.io'

# Configurar los encabezados para simular el comportamiento de un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8'}

# Definir una función que extraiga los datos de un asset de AtomicHub mediante webscraping
def get_asset(page_id):

    # Inicia una instancia del navegador web
    driver = get_driver()

    # Configurar la URL de la página a analizar
    page_url = f'{base_url}{page_id}'

    # Enviar la solicitud HTTP a la página
    response = requests.get(page_url, headers=headers)
    
    # Abrir la página en el navegador
    driver.get(page_url)
    time.sleep(1)
    
    # Verificar si la solicitud se completó con éxito
    if response.status_code != 200:
        print(f'La solicitud no se completó con éxito para el asset {asset_id}. Error: {response.status_code}')

    # Si la solicitud se completó con éxito, extraer la información relevante
    else: 

        # Utilizar Beautiful Soup para analizar el HTML y extraer la información relevante
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extraer el nombre del asset
        name = soup.find('h3', {'data-testid': 'info-card-title'}).text 
        
        # Extraer datos de las tablas
        rows = soup.find('div', {'class': 'row', 'data-testid': 'info-card-content'}).find_all('div', {'class': 'InfoRow row'})
        
        # Extraer el asset_id
        asset_id = rows[0].find('a', href=True)
        asset_id = asset_id.text.replace('#', '')
        
        # Extraer el nombre de la colección
        collection_name = rows[5].find('a', href=True).text
    
        # Extraer vendedor
        seller = rows[1].find('a', href=True).text
        
        # Extraer el nombre del esquema
        schema_name = rows[6].find('a', href=True).text
        
        # Extraer los precios
        prices = soup.find('div', {'class': 'InfoCta__info-container'}).find('span', {'class': 'PriceLabel'}).find('span')
        price = prices.find('span', {'class': 'price-color'}).text
        price_usd = prices.find('span', {'class': 'usd-price'}).text
    
   
        # Cerrar el navegador
        driver.quit()
        
        # Crear un diccionario para almacenar los datos
        asset_data = {
            'name': name,
            'asset_id': asset_id,
            'collection_name': collection_name,
            'seller': seller,
            'schema_name': schema_name,
            'price': price,
            'price_usd': price_usd
        }
    
        # Imprimir mensaje de éxito
        print(f'Datos del asset {asset_id} descargados correctamente')
    
        return asset_data
    
    # Generar una espera aleatoria entre 1 y 5 segundos para simular el comportamiento humano
    sleep_time = random.uniform(1, 5)    
    time.sleep(sleep_time)

In [6]:
# Definir una función para descargar datos de forma concurrenteS
def process_assets(asset_ids):

    # Definir nombres de columnas del dataframe
    column_names = ['Name', 'Asset ID', 'Collection Name', 'Seller', 'Schema Name', 'Price', 'Price USD']
    
    # Crear un DataFrame vacío para almacenar los datos
    df = pd.DataFrame(columns=column_names)

    # Utilizar ThreadPoolExecutor para procesar múltiples assets de forma concurrente
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = executor.map(get_asset, asset_ids)

    # Añadir los resultados al DataFrame
    for result in results:
        if result is not None:
            df.loc[len(df)] = [result['name'], result['asset_id'], result['collection_name'], result['seller'], result['schema_name'], result['price'], result['price_usd']]

    return df

In [7]:
# Procesar los assets y obtener los resultados en un DataFrame
df = process_assets(unique_page_ids)

Datos del asset 1099596675169 descargados correctamente
Datos del asset 1099597285655 descargados correctamente
Datos del asset 1099600692578 descargados correctamente
Datos del asset 1099520858219 descargados correctamente
Datos del asset 1099597863192 descargados correctamente
Datos del asset 1099521183790 descargados correctamente
Datos del asset 1099675776829 descargados correctamente
Datos del asset 1099598474219 descargados correctamente
Datos del asset 1099596764815 descargados correctamente
Datos del asset 1099677746929 descargados correctamente
Datos del asset 1099597304168 descargados correctamente
Datos del asset 1099596983341 descargados correctamente
Datos del asset 1099611647804 descargados correctamente
Datos del asset 1099596712324 descargados correctamente
Datos del asset 1099598064253 descargados correctamente
Datos del asset 1099675953151 descargados correctamente
Datos del asset 1099596712323 descargados correctamente
Datos del asset 1099597002840 descargados correc

In [8]:
# Guardar los datos extraidos mediante webscraping en un archivo CSV
df.to_csv('..\dataset\data-webscraping.csv', index=False)

In [9]:
# Configurar la URL base de assets de la API de AtomicHub
base_url = 'https://wax.api.atomicassets.io/atomicassets/v1/assets/'


# Definir una función que extraiga los datos de un asset de AtomicHub mediante API
def get_asset_api(asset_id):
    
    # Configurar la URL de la página a analizar
    api_url = f'{base_url}{asset_id}'

    # Definir el número de intentos
    attempts = 0

    # Ejecutar un bucle infinito para intentar obtener los datos del asset
    while attempts < 3:
        # Enviar la solicitud HTTP a la página
        response = requests.get(api_url)
        
        # Verificar si la solicitud se completó con éxito
        if response.status_code == 200:
            # Obtener el contenido JSON
            asset_data = response.json()
            break
        else:
            # Imprimir el código de error si la solicitud no se completó con éxito
            print(f'Error: {response.status_code} con el asset {asset_id}')
            attempts += 1
             # Definir tiempo de espera para volver a intentar
            time.sleep(3)
    else:
        # Imprimir el mensaje de error si se alcanzó el límite de intentos
        print(f'Demasiados intentos fallidos con el asset {asset_id}')
        return None

    # Extraer el nombre del asset
    api_name = asset_data['data']['name']
    
    # Extraer el nombre de la colección
    collection_name = asset_data['data']['collection']['collection_name']
    
    # Extraer vendedor
    seller = asset_data['data']['owner']
    
    # Extraer el nombre del esquema
    schema_name = asset_data['data']['schema']['schema_name']
    
    # Crear dataframe para almacenar los datos
    df_api.loc[len(df_api)] = [api_name,
                               asset_id,
                               collection_name,
                               seller,
                               schema_name]
    
    # Imprimir mensaje de éxito
    print(f'Datos del asset {asset_id} descargados correctamente')
    
    # Definir tiempo de espera
    time.sleep(1)

In [10]:
# Definir función que itere la función de extración de datos con API sobre los assets
def map_assets_api(df):
    
    # Iterar la función de extración de datos con API sobre los assets
    list(map(get_asset_api, list(df['Asset ID'])))

In [11]:
# Importar el dataframe con los datos extraidos mediante webscraping
# Descomentar la siguiente línea si se desea importar el dataframe desde un archivo CSV
# df = pd.read_csv('data-webscraping.csv')

In [12]:
# Definir nombres de columnas del dataframe
column_names = ['Name', 'Asset ID', 'Collection Name', 'Seller', 'Schema Name']

# Crear dataframe vacío para los datos extraidos por API
df_api = pd.DataFrame(columns=column_names)

# Ejecutar la función map_assets_api para extraer los datos de los assets mediante API
map_assets_api(df)

Datos del asset 1099597285655 descargados correctamente
Datos del asset 1099600692578 descargados correctamente
Datos del asset 1099596675169 descargados correctamente
Datos del asset 1099520858219 descargados correctamente
Datos del asset 1099597863192 descargados correctamente
Datos del asset 1099521183790 descargados correctamente
Datos del asset 1099675776829 descargados correctamente
Datos del asset 1099598474219 descargados correctamente
Datos del asset 1099596764815 descargados correctamente
Datos del asset 1099677746929 descargados correctamente
Datos del asset 1099597304168 descargados correctamente
Datos del asset 1099596983341 descargados correctamente
Datos del asset 1099611647804 descargados correctamente
Datos del asset 1099675953151 descargados correctamente
Datos del asset 1099596712324 descargados correctamente
Datos del asset 1099598064253 descargados correctamente
Datos del asset 1099596712323 descargados correctamente
Datos del asset 1099597002840 descargados correc

In [13]:
# Guardar los datos extraidos mediante API en un archivo CSV
df_api.to_csv('..\dataset\data-api.csv', index=False)

In [14]:
# Comparar los dataframes
df[['Name', 'Asset ID', 'Collection Name', 'Seller', 'Schema Name']].equals(df_api)

True